# script to aggregate calenviroscreen score to county level

### inputs

In [1]:
data_path = '/Volumes/GoogleDrive/.shortcut-targets-by-id/1-BTh8T0PMwHDs3KZ9V--KPrNWRUgRxvV/2020_CAEECC_Public_Sector_Underserved/data/raw/calenviroscreen'
data_file = 'ces3results.xlsx'

### outputs

In [2]:
save_path = '/Volumes/GoogleDrive/.shortcut-targets-by-id/1-BTh8T0PMwHDs3KZ9V--KPrNWRUgRxvV/2020_CAEECC_Public_Sector_Underserved/data/processed'
save_file = 'ces_dac_county_proportion_median.csv'

### load libraries

In [3]:
import pandas as pd
import os
import glob

## script

### read in calenviroscreen data

In [4]:
df_ces = pd.read_excel(os.path.join(data_path, data_file), sheet_name='CES 3.0 (2018 Update)')

In [5]:
df_ces.head()

,Census Tract,Total Population,California County,ZIP,Nearby City \n(to help approximate location only),Longitude,Latitude,CES 3.0 Score,CES 3.0 Percentile,CES 3.0 \nPercentile Range,...,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6019001100,3174,Fresno,93706,Fresno,-119.781696,36.709695,94.090246,100.000000,95-100% (highest scores),...,77.509665,76.3,97.121307,17.6,91.724838,26.0,79.398324,92.120494,9.553509,99.697314
1,6071001600,6133,San Bernardino,91761,Ontario,-117.618013,34.057780,90.677839,99.987388,95-100% (highest scores),...,96.253833,72.5,94.632307,12.3,71.823836,34.1,93.754760,87.436849,9.067784,98.108210
2,6019000200,3167,Fresno,93706,Fresno,-119.805504,36.735491,85.970036,99.974776,95-100% (highest scores),...,78.389548,86.8,99.560025,16.1,87.980708,40.1,97.854785,94.581328,9.808714,99.987388
3,6077000801,6692,San Joaquin,95203,Stockton,-121.314524,37.940517,82.491521,99.962164,95-100% (highest scores),...,75.136648,61.3,85.568825,19.6,94.973981,21.1,63.544047,86.701266,8.991499,97.717241
4,6019001500,2206,Fresno,93725,Fresno,-119.717843,36.681600,82.030814,99.949552,95-100% (highest scores),...,73.723504,66.4,90.232558,18.6,93.654017,28.1,83.980706,80.075199,8.304332,92.760752


### only keep select few columns

In [6]:
df_ces_sel = df_ces[['Census Tract', 'California County', 'CES 3.0 Score', ' CES 3.0 Percentile', 'SB 535 Disadvantaged Community']]

In [7]:
df_ces_sel = df_ces_sel.rename(columns = {'Census Tract': 'census_tract', 
                                          'California County': 'county',
                                          'CES 3.0 Score': 'ces_score',
                                          ' CES 3.0 Percentile': 'ces_percentile',
                                          'SB 535 Disadvantaged Community': 'dac_flag'})

In [8]:
df_ces_sel.head()

,census_tract,county,ces_score,ces_percentile,dac_flag
0,6019001100,Fresno,94.090246,100.000000,Yes
1,6071001600,San Bernardino,90.677839,99.987388,Yes
2,6019000200,Fresno,85.970036,99.974776,Yes
3,6077000801,San Joaquin,82.491521,99.962164,Yes
4,6019001500,Fresno,82.030814,99.949552,Yes


### aggregate ces scores to county level

count number of dac flagged census tracts within each county

In [9]:
count_dac = df_ces_sel.groupby(['county', 'dac_flag'])['census_tract'].count().reset_index(name = 'counts')

In [10]:
count_dac.head()

,county,dac_flag,counts
0,Alameda,No,322
1,Alameda,Yes,38
2,Alpine,No,1
3,Amador,No,9
4,Butte,No,49


check that there are no counties with zero disadvantaged communities:

In [11]:
count_dac[(count_dac.dac_flag == 'Yes') & (count_dac.counts == 0)]

,county,dac_flag,counts


count how many census tracts there are in each county

In [12]:
count_tracts = df_ces_sel.groupby(['county'])['census_tract'].count().reset_index(name = 'total')

In [13]:
count_tracts.head()

,county,total
0,Alameda,360
1,Alpine,1
2,Amador,9
3,Butte,51
4,Calaveras,10


merge count of dac flags with total count of census tracts

In [14]:
prop_dac = count_dac.merge(count_tracts, on = 'county')

In [15]:
prop_dac.head()

,county,dac_flag,counts,total
0,Alameda,No,322,360
1,Alameda,Yes,38,360
2,Alpine,No,1,1
3,Amador,No,9,9
4,Butte,No,49,51


calculation proportion of dac-assigned census tracts within each county

In [16]:
prop_dac['dac_proportion'] = prop_dac['counts']/prop_dac['total']

In [17]:
prop_dac.head()

,county,dac_flag,counts,total,dac_proportion
0,Alameda,No,322,360,0.894444
1,Alameda,Yes,38,360,0.105556
2,Alpine,No,1,1,1.000000
3,Amador,No,9,9,1.000000
4,Butte,No,49,51,0.960784


keep calculation of dac flag == 'yes' only

In [18]:
prop_dac = prop_dac[(prop_dac.dac_flag == 'Yes') | ((prop_dac.dac_flag == 'No') & (prop_dac.counts == prop_dac.total))]

In [19]:
prop_dac.loc[((prop_dac.dac_flag == 'No') & (prop_dac.counts == prop_dac.total)), 'dac_proportion'] = 0

In [20]:
prop_dac = prop_dac[['county', 'dac_proportion']]

take median of ces score by county

In [21]:
median_ces = df_ces_sel.groupby(['county'])['ces_score', 'ces_percentile'].agg(['median']).reset_index()

In [22]:
median_ces.head()

,county,ces_score,ces_percentile
,,median,median
0,Alameda,21.532906,42.098625
1,Alpine,11.343357,15.853197
2,Amador,23.257549,45.756085
3,Butte,20.033261,38.365494
4,Calaveras,15.180852,25.551772


In [23]:
median_ces.columns = median_ces.columns.map('_'.join).str.strip('_')

In [24]:
median_ces.head()

,county,ces_score_median,ces_percentile_median
0,Alameda,21.532906,42.098625
1,Alpine,11.343357,15.853197
2,Amador,23.257549,45.756085
3,Butte,20.033261,38.365494
4,Calaveras,15.180852,25.551772


In [25]:
# agg_ces.columns = agg_ces.columns.droplevel(0)

combine all county-level variables

In [26]:
agg_ces = prop_dac.merge(median_ces, on = 'county')

In [27]:
agg_ces.head()

,county,dac_proportion,ces_score_median,ces_percentile_median
0,Alameda,0.105556,21.532906,42.098625
1,Alpine,0.000000,11.343357,15.853197
2,Amador,0.000000,23.257549,45.756085
3,Butte,0.039216,20.033261,38.365494
4,Calaveras,0.000000,15.180852,25.551772


### export to csv file

In [28]:
agg_ces.to_csv(os.path.join(save_path, save_file), index=False)